<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [42]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
train_data = pd.read_csv("./train.csv")
test_data = pd.read_csv("./sample_submission.csv")
upper_cases = [chr(ord("A") + val) for val in range(26)]

In [3]:
title_ids = list(set(train_data["Id"]))
id_title_dict = dict()
for tid in title_ids:
    id_title_dict[tid] = pd.read_json(os.path.join("./train/", tid+".json"))
    
# id_title_dict["0007f880-0a9b-492d-9a58-76eb0b0e0bd7"]

In [4]:
def find_is_datalabel(datasets, can_dataset):
    for target in datasets:
        if can_dataset.find(target) > -1 or target.find(can_dataset) > -1:
            return 1 
    return 0

id_datalabel_dict = dict()
for id_, dataset_ in zip(train_data["Id"], train_data["dataset_label"]):
    id_datalabel_dict.setdefault(id_, []).append(dataset_)

    
def find_big_case(word):
    if word.find("CoV") > -1 or word.find("COV") > -1:
        return True
    
    for l in word:
        if l not in upper_cases:
            return False
    
    return (len(word) >= 4)


def find_candidate_sen(text):
#     print(text)
#     text = "69 The Baltimore Longitudinal Study of Aging (BLSA) recently reported that the association between AD-related pathology and IR could not be confirmed."
    text = text.replace("\n", " ")
    words = text.split(" ")
    candidate_sentences = []
    candidate_datasets = []
    sentence = []
    can_dataset = []
    flag = 0
    record = False
    patience = 1
    flag_st = flag_ed = None
    for word_id, word in enumerate(words):
        if find_big_case(word):
#             print(word)
            record = True
            can_dataset.append(word)
#             can_dataset.append(" ".join(words[max(0, word_id-5):word_id+5]))
#             print(" ".join(words[max(0, word_id-5):word_id+5]))
        sentence.append(word)
      
        if len(word) > 0 and word[0] in upper_cases:
            patience = 1
            flag += 1
            if flag == 1:
                flag_st = word_id
#                 print(word)
            if flag == 2:
                record = True
                  
        else:
            if patience == 0:
                if flag > 1:
                    flag_ed = word_id
#                     print(" ".join(words[flag_st:flag_ed]))
                    can_dataset.append(" ".join(words[flag_st:flag_ed]))
                flag = 0
                patience = 1
            else:
                patience -= 1
#         print(word)

        if word.endswith(",") or word.endswith(":") or word.endswith(";"):
            if flag > 1:
                flag_ed = word_id
                can_dataset.append(" ".join(words[flag_st:flag_ed+1]))
            flag = 0
            patience = 1
        if len(word) > 0 and word[-1] in [".", "?", "!"] and (len(word) < 2 or (word[-2] not in upper_cases)):
            
            if record:
                candidate_sentences.append(" ".join(sentence))
                candidate_datasets.append(can_dataset)
            can_dataset = []
            sentence = []
            record = False
            flag = 0
            patience = 1
        
#     s=2
    return candidate_sentences, candidate_datasets

In [5]:
id_dfs_dict = dict()
for tid in tqdm(title_ids):
    dfs = []
    total_row_num = id_title_dict[tid].shape[0]
    for row_id in range(total_row_num):
        candidate_sentences, candidate_datasets = find_candidate_sen(id_title_dict[tid]["text"][row_id])
#         print(candidate_sentences, candidate_datasets)
        if len(candidate_sentences)>0:
            pd_sentences, pd_datasets, labels = [], [], []
            for idx in range(len(candidate_sentences)):
                for dataset_ in candidate_datasets[idx]:
                    pd_sentences.append(candidate_sentences[idx])
                    pd_datasets.append(dataset_)
#                     print(candidate_sentences[idx], dataset_)
            
                    labels.append(find_is_datalabel(id_datalabel_dict[tid], dataset_))
            df = pd.DataFrame()
            df["Label"] = labels
            df["Sentence"] = pd_sentences
            df["Can_data"] = pd_datasets
            df["Section"] = id_title_dict[tid]["section_title"][row_id]
            df["Id"] = tid
            dfs.append(df)
    
    if dfs:
        dfs = pd.concat(dfs, axis=0)  
        id_dfs_dict[tid] = dfs
    else:
        print(tid)

can_dataset_extract = pd.concat(list(id_dfs_dict.values()), axis=0)

sen_len = [len(val.split(" ")) for val in can_dataset_extract["Sentence"]]
can_dataset_extract["sen_length"] = sen_len


 23%|██▎       | 3287/14316 [03:53<14:40, 12.53it/s]

55f1cc31-58e6-4ca0-b8bf-bc80bfecec09


100%|██████████| 14316/14316 [17:11<00:00, 13.89it/s]


In [5]:
# plt.hist(can_dataset_extract["sen_length"], bins=10)
# a = set(can_dataset_extract[can_dataset_extract["sen_length"]>500]["Sentence"].values)
can_dataset_extract = can_dataset_extract[can_dataset_extract["sen_length"]<200]
can_dataset_extract.drop(labels=["Unnamed: 0"], axis=1, inplace=True)
can_dataset_extract.drop_duplicates(inplace=True)


filters = (can_dataset_extract["Label"] == 0).values & (can_dataset_extract["sen_length"]>100).values
can_dataset_extract = can_dataset_extract[~filters]
can_dataset_extract = can_dataset_extract[~can_dataset_extract["Can_data"].isnull()]

In [83]:
# save data

can_dataset_extract.to_csv("data1.csv")

full_data = can_dataset_extract.sample(frac=1)
train_num = int(full_data.shape[0] * 0.8)
train_data = full_data.iloc[:train_num, :]
valid_data = full_data.iloc[train_num:, :]

train_data.to_csv("data1_train.csv")
valid_data.to_csv("data1_val.csv")

print(train_data.shape, (train_data["Label"]==1).mean())
print(valid_data.shape, (valid_data["Label"]==1).mean())

(1392462, 6) 0.04511433705192673
(348116, 6) 0.04532971768031346


In [84]:
train_data

,Label,Sentence,Can_data,Section,Id,sen_length
504601,0.0,The second period mainly includes modern CTD d...,Nordic Seas and,Temporal development,e26d2c87-b41e-44ad-8af8-079648a9bab5,26
26517,0.0,The approximate location of the center of the ...,"Ajith H. Perera,",Spatial variation: habitat disturbances,22be8f26-7e69-4559-bf48-2bd13a5ada35,35
1273254,0.0,Section Three discusses the background of the ...,Section Three discusses,NaN,4f642b3e-da80-4e53-8a86-fde746bb21af,15
773738,0.0,PISA is one of the world's largest educational...,PISA,Introduction,95d21d82-310e-4ce7-9e5a-30bbdec1eb8d,47
943276,0.0,http://dx.doi.org/10.1002/grl.50880 ical Rese...,"Research Atmospheres,",NaN,5de064a4-925f-4076-8f70-4691c0bbf6bf,7
...,...,...,...,...,...,...
1517943,0.0,Since these outcomes are counts truncated at z...,X. I include,III. Estimation Methods,28a1e9e6-9ea5-4a5e-8a33-c62873b71823,58
767236,0.0,Although U.S. data on high-skill migration con...,Although U.S. data,Introduction,d58436ef-1479-4a86-aa71-6dd45e5f24d0,44
1121213,0.0,A simulation model was developed to predict in...,Whitehall II participants,Simulation study,a0014e7b-99a6-4149-984e-79bbe72caafa,27
1439027,0.0,• International Health Terminology Standards D...,International Health Terminology Standards Dev...,C. HEALTHCARE STANDARDS FOR DATA INTEGRATION,3e79dc82-cabb-4b1c-8f08-3e54e6597754,23


In [159]:
# check 
# print(can_dataset_extract["Label"].mean())
# can_dataset_extract[can_dataset_extract["Label"]==1]

# list(set(train_data["dataset_label"]))

In [161]:
# check if omit some data label

# non_found_dataset = []
# for dataset in list(set(train_data["dataset_label"])):
# #     print("======")
# #     print(dataset)
#     flag = 0
#     for val in can_dataset_extract["Can_data"].values:
#         if val.find(dataset) > -1 or dataset.find(val) > -1:
# #             print(val)
#             flag = 1
#             break
#     if flag == 0:
#         non_found_dataset.append(dataset)

# len(non_found_dataset), non_found_dataset

In [166]:
# counted = 0
# for idex, tid in enumerate(title_ids):
#     for val in id_title_dict[tid]["section_title"].values:
#         if str(val).find("Data") > -1 or str(val).find("data") > -1 :
#             counted += 1
#             print(idex)
#             break
# counted

# Bert vectorlize

In [100]:
import torch
import pandas as pd
from transformers import BertTokenizer
from transformers import BertModel
from tqdm import tqdm

In [2]:
can_dataset_extract = pd.read_csv("data1.csv")

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
model = BertModel.from_pretrained("bert-base-cased",output_hidden_states = True,)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [114]:
def find_pos(a, b):
    v = np.ones_like(b)
    conv = np.convolve(a, v, "valid")
    start = np.argmax(conv==b.sum())
#     if (conv==b.sum()).sum() != 1:
#         print(a, b)
    return (start, start + b.shape[0])

# data paperid candidate label
# Tokenize all of the sentences and map the tokens to their word IDs.
input_ids = []
can_dataset_pos = []
attention_masks = []
allow_max_length = 202
# For every sentence...
for idx in range(can_dataset_extract.shape[0]):
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    
    can_dataset = can_dataset_extract["Can_data"].values[idx].split(" ")
    sent = can_dataset_extract['Sentence'].values[idx].split(" ")
    encoded_dict1 = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        is_split_into_words=True,
                        max_length = allow_max_length,           # Pad & truncate all sentences.
                        padding='max_length',
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    encoded_dict2 = tokenizer.encode_plus(
                        can_dataset,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        is_split_into_words=True,
                        max_length = 202,           # Pad & truncate all sentences.
                        padding='max_length',
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )   
    
    
    # Add the encoded sentence to the list.    
    input_id = encoded_dict1['input_ids']
    input_ids.append(input_id)
    
    pattern = encoded_dict2["input_ids"][0]
    pattern = pattern[pattern>0][1:-1]

    pos = find_pos(input_id[0][input_id[0]>0].numpy(), pattern.numpy())
    can_dataset_pos.append(pos)
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])
    
    if idx % 5000 == 0:
        print(idx)

# # Convert the lists into tensors.
# input_ids = torch.cat(input_ids, dim=0)
# attention_masks = torch.cat(attention_masks, dim=0)
# labels = torch.tensor(can_dataset_extract['Label'])

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
235000
240000
245000
250000
255000
260000
265000
270000
275000
280000
285000
290000
295000
300000
305000
310000
315000
320000
325000
330000
335000
340000
345000
350000
355000
360000
365000
370000
375000
380000
385000
390000
395000
400000
405000
410000
415000
420000
425000
430000
435000
440000
445000
450000
455000
460000
465000
470000
475000
480000
485000
490000
495000
500000
505000
510000
515000
520000
525000
530000
535000
540000
545000
550000
555000
560000
565000
570000
575000
580000
585000
590000
595000
600000
605000
610000
615000
620000
625000
630000
635000
640000
645000
650000
655000
660000
665000
670000
675000
680000
685000
690000
695000
700000
705000
710000
715000
720000
725000
730

RuntimeError: Sizes of tensors must match except in dimension 0. Got 202 and 262 in dimension 1 (The offending index is 1359)

In [147]:
new_input_ids = []
new_can_dataset_pos = []
new_attention_masks = []
new_labels = []
allow_max_length = 202

for idx in range(len(input_ids)):
    if input_ids[idx].shape[-1] == allow_max_length:
        new_input_ids.append(input_ids[idx])
        new_can_dataset_pos.append(can_dataset_pos[idx])
        new_attention_masks.append(attention_masks[idx])
        new_labels.append(can_dataset_extract["Label"].values[idx])
        
input_ids = new_input_ids
can_dataset_pos = new_can_dataset_pos
attention_masks = new_attention_masks
labels = new_labels

In [150]:
term_dict = {"id": input_ids,
             "can_dataset_pos": can_dataset_pos, 
             "attention_masks": attention_masks,
             "label": labels}

import pickle
pickle.dump(term_dict, open("term_input_ids.pt", "wb"))






  0%|          | 2019/1729723 [01:07<11:32:50, 41.56it/s]

In [152]:
# run in bash shell

model.eval()
model.to("cuda:0")
candicate_embeddings = []
n = len(input_ids)
with torch.no_grad():  # 将输入传入模型，得到每一层的输出信息，这里的encoded_layers为12层，可以打印验证
    for i in range(n):
        outputs = model(input_ids[i].to("cuda:0"))
        hidden_states = outputs[2]
        
        token_embeddings = torch.cat(hidden_states, dim=0) # 13 * sen_len * 768
        # Swap dimensions 0 and 1.
        token_embeddings = token_embeddings.permute(1, 0, 2)# sen_len * 13 * 768
        # For each token in the sentence...
        token_vecs_sum = torch.sum(token_embeddings[:,-4:,:], dim=1)
        start, end = can_dataset_pos[i]
        candicate_embeddings.append(token_vecs_sum[start:end].cpu().numpy())
            
        if i % 5000 == 0:
            print(i)

In [154]:
term2_dict = {"can_embeddings": candicate_embeddings,
              "labels": labels}

pickle.dump(term2_dict, open("can_embeddings.pt", "wb"))